# TME 2 : Prédiction de gènes



## Partie 1 : Détection de gènes candidats


Le but de cette partie est d’annoter les régions dans le génome qui correspondent à des gènes. Nous allons prendre en compte les différents éléments qui définissent un gène pour pouvoir déterminer les candidats à des régions codantes : phases ouvertes de lecture, propriétés statistiques du code génétique, et comparer les résultats avec l’annotation qui est disponible. 
Nous nous baserons sur le génome de _Bifidobacterium actinocoloniiforme_ que nous avons commencé à analyser au TME précédent, avec ses annotations et au génome de _Escherichia coli_


In [1]:
import doctest # C’est pouvoir utiliser doctest.testmod()

+ (mise en route) Adapter les commandes du TME précédent pour télécharger le génome de _E. coli_ ainsi que ses annotations

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz


In [2]:
# Dictionnaire codons -> acides aminés. Les codons stops sont représentés
# avec le caractère "*"
CODEGENETIQUE = {
    "TTT": "F",
    "TTC": "F",
    "TTA": "L",
    "TTG": "L",
    "TCT": "S",
    "TCC": "S",
    "TCA": "S",
    "TCG": "S",
    "TAT": "Y",
    "TAC": "Y",
    "TAA": "*",
    "TAG": "*",
    "TGT": "C",
    "TGC": "C",
    "TGA": "*",
    "TGG": "W",
    "CTT": "L",
    "CTC": "L",
    "CTA": "L",
    "CTG": "L",
    "CCT": "P",
    "CCC": "P",
    "CCA": "P",
    "CCG": "P",
    "CAT": "H",
    "CAC": "H",
    "CAA": "Q",
    "CAG": "Q",
    "CGT": "R",
    "CGC": "R",
    "CGA": "R",
    "CGG": "R",
    "ATT": "I",
    "ATC": "I",
    "ATA": "I",
    "ATG": "M",
    "ACT": "T",
    "ACC": "T",
    "ACA": "T",
    "ACG": "T",
    "AAT": "N",
    "AAC": "N",
    "AAA": "K",
    "AAG": "K",
    "AGT": "S",
    "AGC": "S",
    "AGA": "R",
    "AGG": "R",
    "GTT": "V",
    "GTC": "V",
    "GTA": "V",
    "GTG": "V",
    "GCT": "A",
    "GCC": "A",
    "GCA": "A",
    "GCG": "A",
    "GAT": "D",
    "GAC": "D",
    "GAA": "E",
    "GAG": "E",
    "GGT": "G",
    "GGC": "G",
    "GGA": "G",
    "GGG": "G"
}

# Utiliser les sequences TESTSEQ et TESTSEQCLEAN pour tester vos fonctions
TESTSEQ = "ATGAAACGCATTAGCMMCACCATTACCACCACCATCACCATTACCACAGKTAACGGTGCGGGCTGA"
TESTSEQCLEAN = "ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA"

### A) Annotation des phases ouvertes de lecture et traduction en séquence protéique

<b>Question 1)</b> Parfois dans les génomes (représentés par des fichiers numériques), nous trouvons des nucléotides qui n'ont pas été correctement identifiés par la machine de séquençage. Certains instruments nous restreignent les possibilités à un sous-ensemble de nucléotides. Ecrire une fonction `remplace_non_identifies` qui remplace les nucléotides non identifiés par une des possibilités listées ci-dessous de façon aléatoire.<br>

R = G,A (purine)<br>
Y = T,C (pyrimidine)<br>
K = G,T (céto)<br>
M = A,C (amino)<br>
S = G,C (obligations solides)<br>
W = A,T (Les liaisons faibles)<br>
B = G,T,C (tous sauf A)<br>
D = G,A,T (tous sauf C)<br>
H = A,C,T (tous sauf G)<br>
V = G,C,A (tous sauf T)<br>
N = A,G,C,T (any)<br>
X = A,G,C,T (any)<br>



In [3]:
# Question 1
def remplace_non_identifies(seq):
    """
    Remplace les nucléotides non identifiés par une des possibilités de façon aléatoire.
    
    >>> clean = remplace_non_identifies('ATGAAACGCATTAGCMMCACCATTACCACCACCATCACCATTACCACAGKTAACGGTGCGGGCTGA')
    >>> 'M' in clean
    False
    >>> 'K' in clean
    False
    >>> len(clean) == 66
    True
    >>> isinstance(clean, str)
    True
    """
    pass


doctest.testmod()

remplace_non_identifies(TESTSEQ)

**********************************************************************
File "__main__", line 7, in __main__.remplace_non_identifies
Failed example:
    'M' in clean
Exception raised:
    Traceback (most recent call last):
      File "c:\users\diego\appdata\local\programs\python\python37-32\lib\doctest.py", line 1329, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.remplace_non_identifies[1]>", line 1, in <module>
        'M' in clean
    TypeError: argument of type 'NoneType' is not iterable
**********************************************************************
File "__main__", line 9, in __main__.remplace_non_identifies
Failed example:
    'K' in clean
Exception raised:
    Traceback (most recent call last):
      File "c:\users\diego\appdata\local\programs\python\python37-32\lib\doctest.py", line 1329, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.remplace_non_identifies[2]>", line 1, in <module>
        'K' in clean
    Typ

<b>Question 2)</b> Ecrire une fonction `listecodon` qui renvoie une liste de codons pour une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``[‘AAC’, ‘GTG’, ‘GCA’]``. Si la longueur de la séquence n'est pas un multiple de 3 on ne tiendra pas compte des 1 ou 2 nucléotides restant à la fin.


In [ ]:
# Question 2
def listecodon(seq):
    """
    Renvoie une liste de codons pour une séquence passée en paramètre.
    
    Si la longueur de la séquence n'est pas un multiple de 3 elle ne tiendra pas 
    compte des 1 ou 2 nucléotides restant à la fin.
    
    >>> listecodon('AAACCC')
    ['AAA', 'CCC']
    >>> listecodon('AAACC')
    ['AAA']
    >>> listecodon('AAAC')
    ['AAA']
    """
    pass


doctest.testmod()

assert listecodon(TESTSEQCLEAN) == [
    'ATG', 'AAA', 'CGC', 'ATT', 'AGC', 'ACC', 'ACC', 'ATT', 'ACC', 'ACC',
    'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'GGT', 'GCG',
    'GGC', 'TGA'
]

<b>Question 3)</b> Ecrire une fonction `reversecompl` qui renvoie le brin complémentaire d’une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``TGCCACGTT``.


In [ ]:
# Question 3
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence."""
    pass


assert reversecompl(
    TESTSEQCLEAN
) == "TCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCAT"

<b>Question 4)</b> Ecrire une fonction `liste_orfs` qui retourne la liste de tous les cadres ouverts de lectures, c'est à dire les séquence commençant par un des codons start: ``ATG``, ``GTG``, ``TTG`` (``ATG`` est le plus fréquent) et finissant par un des codons stop : ``TAA``, ``TAG``, ``TGA``. Vous renverrez les séquences pour les 6 phases de lecture (3 pour le brin sens et 3 pour le brin complémentaire). Ces séquences sont nommées séquences CDS (pour _CoDing Sequences_ en anglais) et les phases ouvertes de lecture ORFS (pour _Open Reading Frame_). <br>
Note 1 : si plusieurs start sont trouvés dans la même phase de lecture qu'un stop et en amont de celui ci, le CDS débute par conventions au start le plus en amont. <br> 
Note 2 : les CDS peuvent se chevaucher (sur différentes phases et éventuellement sur le brin d'ADN complémentaire).  
Note 3 : le function doit aussi garder le brin et la commence et la fin du CDS.


In [ ]:
# Question 4
def liste_orfs(seq):
    """
    Retourne la liste de tous les cadres ouverts de lectures.
    
    >>> sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    [(-1, 0, 9, 'ATGCCCTAA'), (1, 2, 11, 'ATGATGTAA'), (1, 13, 22, 'GTGTTTTGA')]
    """
    pass

<b>Question 5)</b> Développez une fonction `traduit_prot` qui étant donné un gène renvoie la protéine traduite. Utilisez le dictionnaire définit ci dessous.

In [ ]:
# Question 5
def traduit_prot(seq, codegenetique=CODEGENETIQUE):
    """Renvoie la protéine traduite, par défaut il utilise CODEGENETIQUE."""
    pass


assert traduit_prot(TESTSEQCLEAN) == "KRISTTITTTITITTGNGAG*"